# Model Context Protocol (MCP) on Databricks

MCP is an open source standard that connects AI agents to tools, resources, prompts, and other contextual information.
Databricks provides the following types of MCP servers:
1. **Managed MCP:** Immediately access Databricks features using pre-configured MCP servers.
2. **External MCP:** Securely connect to MCP servers hosted outside of Databricks using managed connections.
3. **Custom MCP:** Host a custom MCP server as a Databricks App. 

In this notebook, you'll see how to build and use different types of MCP servers that can be later used as tools in our customer service agent:
1. Managed MCP: Build a **Unity Catalog Function** using SQL (or Python) callable by your agents. 
2. External MCP: Connect to [**GitHub MCP server**](https://github.com/github/github-mcp-server) using Unity Catalog Connections. 
3. Custom MCP: Build and host a **custom MCP server as a Databricks App.**

## 1. Managed MCP Servers
Databricks provides the following MCP servers that work out of the box.


| MCP server | URL pattern |
|---|---|
| **Vector search:** Query Vector Search indexes to find relevant documents. | https://workspace-hostname/api/2.0/mcp/vector-search/{catalog}/{schema}/{index_name} |
| **Genie space:** Query Genie spaces to analyze structured data using natural language. | https://workspace-hostname/api/2.0/mcp/genie/{genie_space_id} |
| **DBSQL:** Run AI-generated SQL to author data pipelines with AI coding tools such as Claude Code, Cursor, and Codex. | https://workspace-hostname/api/2.0/mcp/sql |
| **Unity Catalog functions:** Use Unity Catalog functions to run predefined SQL queries. | https://workspace-hostname/api/2.0/mcp/functions/{catalog}/{schema}/{function_name} |

### **Data Setup**

In [0]:
%run ./config

In [0]:
%python
dbutils.widgets.text("catalog_name", catalog_name)
dbutils.widgets.text("schema_name", schema_name)

---

### **Unity Catalog Functions**

#### Use the User Name to Look Up the Order History
- **Action**: Query your order management system or customer database using the Username.  
- **Why**: Reviewing past purchases, return patterns, and any specific notes helps you determine appropriate next steps (e.g., confirm eligibility for return).

###### Note: We're doing a little trick to give the LLM extra context into the current date by adding todays_date in the function's response
---

In [0]:
%sql
CREATE OR REPLACE FUNCTION
  IDENTIFIER(:catalog_name || '.' || :schema_name || '.get_order_history')(user_name STRING)
RETURNS TABLE (returns_last_12_months INT, issue_category STRING, todays_date DATE)
COMMENT 'This takes the user_name of a customer as an input and returns the number of returns and the issue category'
LANGUAGE SQL
RETURN 
SELECT 
  COUNT(*) as returns_last_12_months, 
  issue_category, 
  now() as todays_date
FROM IDENTIFIER('${catalog_name}.${schema_name}.cust_service_data')
WHERE name = user_name 
GROUP BY issue_category;

In [0]:
%sql
SELECT * 
FROM IDENTIFIER(:catalog_name || '.' || :schema_name || '.get_order_history')('Michelle Martin')

### External MCP Server
Connect agents to third-party Model Context Protocol (MCP) servers through Databricks-managed proxies to access external tools and services.

[Databricks Marketplace](https://docs.databricks.com/aws/en/generative-ai/mcp/external-mcp?language=From+Databricks+Marketplace) hosts many commonly used MCP servers. If not available in Marketplace, you can create a [Unity Catalog HTTP connection](https://docs.databricks.com/aws/en/generative-ai/mcp/external-mcp?language=Custom+HTTP+connection) to install any MCP server, including self-hosted or third-party servers. 

In this example, we'll create an HTTP connection to a GitHub hosted server to create issues when a user reports a bug. 

### Host a custom MCP server using Databricks Apps
Host custom or third-party MCP servers as [Databricks apps](https://docs.databricks.com/aws/en/dev-tools/databricks-apps/). Custom MCP servers are useful if you already have an MCP server you want to deploy or if you want to run a third-party MCP server as a source of tools.

See [GitHub - custom MCP server template](https://github.com/databricks/app-templates/tree/main/mcp-server-hello-world) for an example of writing your own MCP server and deploying it as a Databricks app.

In this example, we've created a custom MCP server that retrieves latest company return policies and invoice information to check eligibility on customer requests. 
The source code is located in the teclo-custom-mcp-server folder. Note that we've reused our query from AI functions section to create the tool to capture invoice information. You can use this folder to create your Databricks App and host it. 


To see your available MCP servers, go to your workspace > Agents > MCP Servers:
![](https://docs.databricks.com/aws/en/assets/images/agents-mcp-tab-68ec7243c3c18635096e39225ed91f17.png)

### AI Playground

To tie all the MCP tools together, let's open AI Playground and add all the MCP tools: Under 'Tools' -> + Add tool -> MCP Servers
1. **Unity Catalog Function** -> Browse and select the schema under which you created your function from the previous cell in this notebook
2. **External MCP Servers** -> Select the HTTP connection you created
3. **MCP Servers on Databricks Apps** -> Select the app you created with all the tools 

You can ask the following questions:
1. Get me the latest customer interaction from the queue
2. Whats the company return policy?
3. Based on invoice date and customer order history, is the customer eligible for return or not?
4. Can you create an issue on <repo-name> stating issues with customer's device?
